In [8]:
import scrapy, requests
from scrapy.http import TextResponse

### scrapy

- 비동기 방식으로 데이터를 수집 > 데이터 수집 속도가 빠름
- 예제: Gmarket 베스트 200 상품 데이터 수집
        1. 스크래피 프로젝트 생성
        2. xpath 찾기 : 링크, 상세 페이지의 데이터
        3. items.py : 코드 작성 : model(데이터의 구조: 수집할 데이터의 컬럼을 정의)
        4. spider.py : 코드 작성 : 크롤링 절차 정의
        5. 스크래피 프로젝트 실행

In [9]:
# 1. 스크래피 프로젝트 생성
!scrapy startproject gmarket

Error: scrapy.cfg already exists in /Users/cyeongy/PycharmProjects/pythonProject/web_crawling/day3/practice/gmarket


In [10]:
!tree gmaket / f

"""
items.py : 수집할 데이터의 구조 정의
middlewares.py : 데이터를 수집할 때 headers 정보와 같은 내용을 설정
pipelines.py : 데이터를 수집한 후의 코드 실행 정의
settings.py : 크롤링 설정 : 크롤링 절차 정의
spiders : 디렉토리 : 크롤링 절차 정의
"""

gmaket  [error opening dir]
/f  [error opening dir]

0 directories, 0 files


'\nitems.py : 수집할 데이터의 구조 정의\nmiddlewares.py : 데이터를 수집할 때 headers 정보와 같은 내용을 설정\npipelines.py : 데이터를 수집한 후의 코드 실행 정의\nsettings.py : 크롤링 설정 : 크롤링 절차 정의\nspiders : 디렉토리 : 크롤링 절차 정의\n'

In [11]:
# 2.xpath 찾기 : 링크, 상세 페이지의 데이터
# 링크 데이터 수집

url = 'http://corners.gmarket.co.kr/Bestsellers'

In [12]:
request = requests.get(url)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

In [21]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953126611765204',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953126611765204']

In [23]:
# 상세 페이지 데이터 수집

link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

In [25]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()').extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()').extract()
title, price

(['브라운 면도기 세정액 CCR5+1 '], ['25,030'])

In [27]:
# 3. items.py : 코드 작성 : model(데이터의 구조: 수집할 데이터의 컬럼을 정의)


In [49]:
%%writefile gmarket/gmarket/items.py
import scrapy


class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [40]:
# 4. spider.py : 코드 작성 : 크롤링 절차 정의

In [52]:
%%writefile gmarket/gmarket/spiders/spider.py

import scrapy
from gmarket.items import GmarketItem


class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ['http://corners.gmarket.co.kr/Bestsellers']

    def parse_content(self, response):
            item = GmarketItem()
            item['title'] = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()').extract()
            item['price'] = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()').extract()
            item['link'] = response.url
            yield item

    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)


Overwriting gmarket/gmarket/spiders/spider.py


In [42]:
g# 5. 스크래피 프로젝트 실행
# gmarket 디렉토리
# scrapy crawl GMB -o items.csv

In [43]:
!scrapy crawl GMB -o items.csv

Scrapy 2.6.2 - no active project

Unknown command: crawl

Use "scrapy" to see available commands
